In [ ]:
#Mount the drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Import required libraries

import cv2
import os
import copy
import pandas as pd
import numpy as np
import matplotlib as plt
from collections import deque
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils import np_utils
%matplotlib inline 

Temporal Length - corresponds to the number of continuous frames you want to have in a single sample.

Temporal Stride - Number of frames that you want to skip b/w two samples

Yield Keyword: https://www.geeksforgeeks.org/python-yield-keyword/

In [ ]:
#Function to yield a single sample.

def file_generator(data_path,data_files,temporal_stride=1,temporal_length=16):

    #data files contains list of all the csv 


    # read all the csv files (one csv file corresponds to one vdieo) in data_files one by one

    for f in data_files: 

      #f corresponds to a particular csv (containing path of frames of that particular video)

        tmp_df = pd.read_csv(os.path.join(data_path,f))
        
        #Get the label for this video/csv
        label_list = list(tmp_df['Label'])  
        
        #Total number of frames in this video
        total_images = len(label_list) 

        if total_images>=temporal_length: 
           #If the number of frames in the video is greater tha temporal length, use that video
            num_samples = int((total_images-temporal_length)/temporal_stride)+1
            print ('num of samples from vid seq-{}: {}'.format(f,num_samples))
            img_list = list(tmp_df['FileName'])

        else:
            #If total number of frames is less than temporal length than skip this video 
            print ('num of frames is less than temporal length; hence discarding this file-{}'.format(f))
            continue

        start_frame = 0
        #Initliaze a queue to store the frames
        samples = deque()  

        #A counter to count the number of samples. one sample has as many frames as defined by temporal length
        samp_count=0 
        for img in img_list:
            samples.append(img)
            if len(samples)==temporal_length: #if the queue has as many frames as temporal length, return it as one sample
                samples_c=copy.deepcopy(samples) # copy the queue as in the next stage frames would be popped
                samp_count+=1
                
                #Pop out as many frames as described by the stride from the left to accomodate new frames
                for t in range(temporal_stride): 
                    samples.popleft()
                yield samples_c,label_list[0]
        
                # return a sample(consisting of as many frames as defined by temporal length) 
                                                # and its corsponding label 

In [ ]:
# Load the samples and their corresponding label for each video

def load_samples(data_cat='train',temporal_stride=1,temporal_length=16):

    #data_cat= cahnge according to your folder names

    #Path to folder containing csvs (Either test or train)
    data_path = os.path.join('data_files',data_cat)

    #List of CSVs
    data_files = os.listdir(data_path)

    # define a generator to read the samples
    file_gen = file_generator(data_path,data_files,temporal_stride,temporal_length)
    iterator = True
    data_list = []

    while iterator:

        try:
            x,y = next(file_gen)
            x=list(x)
            data_list.append([x,y])

        except Exception as e:
            print ('the exception: ',e)
            iterator = False
            print ('end of data generator')
    return data_list

In [ ]:
#Function to shuffle data
def shuffle_data(samples):
    data = shuffle(samples,random_state=2)
    return data

In [ ]:
#Change according to requirements
def preprocess_image(img):
    img = cv2.resize(img,(224,224))
    img = img/255
    return img

In [ ]:
#Creates generator to be fed to the architecture.

def data_generator(data,batch_size=16,temporal_padding='same',shuffle=True):              
  
    # Yields the next training batch.
    # data is an array [[img1_filename,img2_filename...,img16_filename],label1], [image2_filename,label2],...].
   
    #Total number of samples in your data
    num_samples = len(data)

    if shuffle:
        data = shuffle_data(data)

    while True:

        for offset in range(0, num_samples, batch_size):
            print ('Starting index: ', offset) 

            # Get the samples you'll use in this batch
            batch_samples = data[offset:offset+batch_size]

            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            # For each example

            # Loop over every baprint ('Total number of train samples:',len(train_data))tch
            for batch_sample in batch_samples: 
                # Load image (X)

                #x is address of frame
                x = batch_sample[0]

                #y is the label
                y = batch_sample[1]

                temp_data_list = []

                for img in x:
                    try:
                        img = cv2.imread(img)
                        #apply any kind of preprocessing here
                        #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                        img = preprocess_image(img)
                        temp_data_list.append(img)

                    except Exception as e:
                        print (e)
                        print ('error reading file: ',img)

                # Read label (y)
                # Append this sample to your batch
                X_train.append(temp_data_list)
                y_train.append(y)
    
            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            #X_train = np.rollaxis(X_train,1,4)
            y_train = np.array(y_train)

            # # convert to one hot encoding for training keras model
            # y_train = np_utils.to_categorical(y_train, 3)
    
            # yield the next training batch            
            yield X_train, y_train


**Load the training data.**

In [ ]:
train_data = load_samples(data_cat='train',temporal_stride=4,temporal_length=16)

In [ ]:
print ('Total number of train samples:',len(train_data))

**Load the testing data.**

In [ ]:
test_data = load_samples(data_cat='test',temporal_stride=4, temporal_length=16)

In [ ]:
train_generator = data_generator(train_data,batch_size=4,shuffle=True)